In [ ]:
import pandas as pd
import numpy as np
import joblib
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Descriptors import *
from rdkit.Chem.rdMolDescriptors import *
from rdkit.Chem.Lipinski import *
from rdkit.Chem.EState import *
from rdkit.Chem.GraphDescriptors import *
from rdkit.Chem.Graphs import *

def flatten_list(lst):
    flat_list = []
    for item in lst:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))
        else:
            flat_list.append(item)
    return flat_list


ni = 0

def properties_array(sSmiles):
    try:
        m = Chem.MolFromSmiles(sSmiles)
        p1 = AllChem.GetMorganFingerprintAsBitVect(m, 2, 512)
        p2 = Chem.rdMolDescriptors.GetMACCSKeysFingerprint(m)

        p3 = [1000*FpDensityMorgan1(m), 1000*FpDensityMorgan2(m), 1000*FpDensityMorgan3(m), ExactMolWt(m), HeavyAtomMolWt(m), 1000*MaxAbsPartialCharge(m), 1000*MaxPartialCharge(m), 1000*MinAbsPartialCharge(m), 1000*MinPartialCharge(m), NumRadicalElectrons(m), NumValenceElectrons(m),1000*CalcFractionCSP3(m),10*CalcKappa1(m),10*CalcKappa2(m),10*CalcKappa3(m),CalcLabuteASA(m),CalcNumAliphaticCarbocycles(m),CalcNumAliphaticHeterocycles(m),CalcNumAliphaticRings(m),CalcNumAmideBonds(m),CalcNumAromaticCarbocycles(m),CalcNumAromaticHeterocycles(m),CalcNumAromaticRings(m),CalcNumAtomStereoCenters(m),CalcNumBridgeheadAtoms(m),CalcNumHBA(m),CalcNumHBD(m),CalcNumHeteroatoms(m),CalcNumHeterocycles(m),CalcNumLipinskiHBA(m),CalcNumLipinskiHBD(m),CalcNumRings(m),CalcNumRotatableBonds(m),CalcNumSaturatedCarbocycles(m),CalcNumSaturatedHeterocycles(m),CalcNumSaturatedRings(m),CalcNumSpiroAtoms(m),CalcNumUnspecifiedAtomStereoCenters(m),CalcTPSA(m)]
        pa3 = np.array(p3, dtype=np.int16)
        
        p4 = [HeavyAtomCount(m), NHOHCount(m), NOCount(m),NumHAcceptors(m), NumHDonors(m), Chi0(m), Chi1(m)]
        
        p5 = [rdMolDescriptors.BCUT2D(m)]

        pa1 = np.array(list(p1), dtype=np.int16)
        pa2 = np.array(list(p2), dtype=np.int16)
        pa0 = np.concatenate([pa1, pa2])
        pa4 = np.array(p4, dtype=np.int16)
        pa5 = np.array(flatten_list(p5), dtype=np.int16)
        
        pa = np.concatenate([pa0,pa3, pa4,pa5])
        #print(len(pa))

        pa = np.array(pa)

        return pa, True
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None, False

model_qy = joblib.load('./Models/QY_random.pkl') 
scaler_qy = joblib.load('./Models/scaler_model-QY.pkl')

df = pd.read_csv("DATA/DB.csv")

molecules_without_qy = []

for i, row in df.iterrows():
    if pd.isna(row['Quantum yield']): 
        sSmiles = row['Chromophore']
        sSmilesSolvent = row['Solvent']
        
        pa, valid1 = properties_array(sSmiles)
        pa2, valid2 = properties_array(sSmilesSolvent)

        if valid1 and valid2:
            features = np.concatenate([pa, pa2]).reshape(1, -1)
            features_scaled_qy = scaler_qy.transform(features)
            predicted_qy = model_qy.predict(features_scaled_qy)[0]
            
            molecules_without_qy.append({
                'Chromophore': sSmiles,
                'Solvent': sSmilesSolvent,
                'QY': round(predicted_qy, 3)
            })

df_without_qy = pd.DataFrame(molecules_without_qy)

df_without_qy.to_csv("molecules_without_qy.csv", index=False)

print(f"{len(molecules_without_qy)} moléculas sem QY foram salvas no novo CSV.")


In [ ]:
def flatten_list(lst):
    flat_list = []
    for item in lst:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))
        else:
            flat_list.append(item)
    return flat_list


ni = 0

def properties_array(sSmiles):
    try:
        m = Chem.MolFromSmiles(sSmiles)
        p1 = AllChem.GetMorganFingerprintAsBitVect(m, 2, 512)
        p2 = Chem.rdMolDescriptors.GetMACCSKeysFingerprint(m)

        p3 = [1000*FpDensityMorgan1(m), 1000*FpDensityMorgan2(m), 1000*FpDensityMorgan3(m), ExactMolWt(m), HeavyAtomMolWt(m), 1000*MaxAbsPartialCharge(m), 1000*MaxPartialCharge(m), 1000*MinAbsPartialCharge(m), 1000*MinPartialCharge(m), NumRadicalElectrons(m), NumValenceElectrons(m),1000*CalcFractionCSP3(m),10*CalcKappa1(m),10*CalcKappa2(m),10*CalcKappa3(m),CalcLabuteASA(m),CalcNumAliphaticCarbocycles(m),CalcNumAliphaticHeterocycles(m),CalcNumAliphaticRings(m),CalcNumAmideBonds(m),CalcNumAromaticCarbocycles(m),CalcNumAromaticHeterocycles(m),CalcNumAromaticRings(m),CalcNumAtomStereoCenters(m),CalcNumBridgeheadAtoms(m),CalcNumHBA(m),CalcNumHBD(m),CalcNumHeteroatoms(m),CalcNumHeterocycles(m),CalcNumLipinskiHBA(m),CalcNumLipinskiHBD(m),CalcNumRings(m),CalcNumRotatableBonds(m),CalcNumSaturatedCarbocycles(m),CalcNumSaturatedHeterocycles(m),CalcNumSaturatedRings(m),CalcNumSpiroAtoms(m),CalcNumUnspecifiedAtomStereoCenters(m),CalcTPSA(m)]
        pa3 = np.array(p3, dtype=np.int16)
        
        p4 = [HeavyAtomCount(m), NHOHCount(m), NOCount(m),NumHAcceptors(m), NumHDonors(m), Chi0(m), Chi1(m)]
        
        p5 = [rdMolDescriptors.BCUT2D(m)]

        pa1 = np.array(list(p1), dtype=np.int16)
        pa2 = np.array(list(p2), dtype=np.int16)
        pa0 = np.concatenate([pa1, pa2])
        pa4 = np.array(p4, dtype=np.int16)
        pa5 = np.array(flatten_list(p5), dtype=np.int16)
        
        pa = np.concatenate([pa0,pa3, pa4,pa5])
        #print(len(pa))

        pa = np.array(pa)

        return pa, True
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None, False

df = pd.read_csv("DATA/DB.csv")

molecules_without_wl = []

model_wl = joblib.load('./Models/WL_random.pkl')  
scaler_wl = joblib.load('./Models/scaler_modelwl.pkl')

for i, row in df.iterrows():
    if pd.isna(row['Emission max (nm)']):
        sSmiles = row['Chromophore']
        sSmilesSolvent = row['Solvent']
        
        pa, valid1 = properties_array(sSmiles)
        pa2, valid2 = properties_array(sSmilesSolvent)

        if valid1 and valid2:
            features = np.concatenate([pa, pa2]).reshape(1, -1)
            features_scaled_wl = scaler_wl.transform(features)
            predicted_wl = model_wl.predict(features_scaled_wl)[0]
            
            molecules_without_wl.append({
                'Chromophore': sSmiles,
                'Solvent': sSmilesSolvent,
                'Emission max (nm)': round(predicted_wl, 3)
            })

df_without_wl = pd.DataFrame(molecules_without_wl)

df_without_wl.to_csv("Only_wl_data_ML.csv", index=False)

print(f"{len(molecules_without_wl)} moléculas sem WL foram salvas no novo CSV.")
